![TraderPy Logo](https://traderpy.files.wordpress.com/2021/01/cropped-logo3.png)

# Save your Data in CSV and Databases (SQLite)

**Author**: TraderPy - Algorithmic Trading (Tu Khac Nguyen)

**Youtube**: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ


## DISCLAIMER

Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

---

## Getting Stock Data from Broker MT5

In [ ]:
############### Export STOCKs from MT5 ###############
# import libraries
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from IPython.display import display

# connecting to MetaTrader5
# mt5.initialize()
# if not mt5.initialize(path='C:\\Program Files\\Top Trader MetaTrader 5\\terminal64.exe'):
if not mt5.initialize(path='C:\\Program Files\\Top Trader MetaTrader 5-01\\terminal64.exe'):
# if not mt5.initialize(path='C:\\Program Files\\MetaTrader 5 IC Markets (SC)\\terminal64.exe'):
    print("initialize failed, Error code =", mt5.last_error())
    mt5.shutdown()

account="HIDDEN"
authorized=mt5.login(account, server="HIDDEN")

# requesting data
# ohlc = mt5.copy_rates_range("ADVANC", 
#                             mt5.TIMEFRAME_D1, 
#                             datetime(2021, 1, 1),
#                             datetime.now())

# show ohlc data
# display(ohlc[:10])

# convert to DataFrame
# ohlc_df = pd.DataFrame(ohlc)
# ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
# ohlc_df.drop(['tick_volume', 'spread'], inplace=True, axis=1)
# ohlc_df.rename(columns = {'time':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'real_volume':'Volume'}, inplace = True)
# ohlc_df['Adj Close'] = ohlc_df['Close'].copy()
# ohlc_df = ohlc_df[['Date','Open','High','Low','Close','Adj Close','Volume']]

# show DataFrame
# display(ohlc_df)

# get number of symbols with symbols_total()
# num_symbols = mt5.symbols_total()
# num_symbols

# get symbol specifications
# symbol_info = mt5.symbol_info("ADVANC")._asdict()
# symbol_info

# get all symbols and their specifications
symbols = mt5.symbols_get()
# for symbols in symbols:
counter = 0
for symbol in symbols[0:800] :
    counter+=1
    print(f'>>> {counter}. Reading price data of {symbol.name}')
    # requesting data
    ohlc = mt5.copy_rates_range(symbol.name, 
                            mt5.TIMEFRAME_D1, 
                            datetime(2020, 1, 1),
                            datetime.now())
    # convert to DataFrame
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
    ohlc_df.drop(['tick_volume', 'spread'], inplace=True, axis=1)
    ohlc_df.rename(columns = {'time':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'real_volume':'Volume'}, inplace = True)
    ohlc_df['Adj Close'] = ohlc_df['Close'].copy()
    ohlc_df = ohlc_df[['Date','Open','High','Low','Close','Adj Close','Volume']]
    
    # save as csv using df.to_csv(filename.csv)
    print(f'       Exporting {symbol.name}.csv')
    if symbol.name=='COM7':
        ohlc_df.to_csv('COM7_'+'.csv', index=False)
    else:
        ohlc_df.to_csv(symbol.name+'.csv', index=False)
    print('       done...')
    print('-------------------------------------------')
print('<<< COMPLETED !!! >>>')



## Getting Stock Data from MongoDB

In [1]:
############### Export STOCKs from MongoDB ###############
# import libraries
import pymongo
import pandas as pd
from datetime import timezone, datetime, timedelta, tzinfo
import os

# connecting to Pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb     = myclient["stock"]

price1d  = mydb.get_collection('price1ds')

# cursor = price1d.aggregate([
#     {
#         "$group": {
#             "_id":{
#                 "name"   : "$name",
#                 # "ncount" : {
#                 #     "$sum": 1
#                 # }
#             }
#         }
#     },
#     # {
#     #     "$project": {
#     #         "name": 1
#     #     }
#     # },
# ])

cursor = price1d.aggregate([
    {
        '$group': {
            '_id': {
                'name': '$name'
            }, 
            'namecount': {
                '$sum': 1
            }
        }
    }
])


data = pd.DataFrame(cursor)
df   = pd.DataFrame([x for x in data['_id']])['name']

# cursor = price1d.find({}, {"_id: 0", "name: 1"})      // Select * from price1d and show only column 'name'

# for doc in cursor:
#     dt_obj = datetime.fromtimestamp(int(doc.get('time'))).strftime('%d-%m-%Y')
#     # print(dt_obj, doc.get('open'), doc.get('high'), doc.get('low'), doc.get('close'), doc.get('volume'))

counter = 1
for ticker in df:
    # print(ticker)
    print(f'>>> {counter}. Reading price data of {ticker}')
    # requesting data
    query  = {'name': ticker}
    ohlc   = price1d.find(query).sort('time', 1)
    # dt_obj = datetime.fromtimestamp(int(ohlc.get('time'))).strftime('%d-%m-%Y')
    # for i in ohlc:
    #     print(i)

    # convert to DataFrame
    ohlc_df = pd.DataFrame(ohlc)
    
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
    ohlc_df['time'] = ohlc_df['time'].dt.strftime('%Y-%m-%d')
    ohlc_df.drop(['_id', '__v', 'createdAt', 'updatedAt', 'symbols'], inplace=True, axis=1)

#     ohlc_df.drop(['tick_volume', 'spread'], inplace=True, axis=1)
    ohlc_df.rename(columns = {'time':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'volume':'Volume'}, inplace = True)
    ohlc_df['Adj Close'] = ohlc_df['Close'].copy()
    ohlc_df = ohlc_df[['Date','Open','High','Low','Close','Adj Close','Volume']]
    
    # print(ohlc_df)

#     # save as csv using df.to_csv(filename.csv)
    print(f'       Exporting {ticker}.csv')
    if ticker=='COM7':
        ohlc_df.to_csv('COM7_'+'.csv', index=False)
        # print('COM7 !!!')
    else:
        ohlc_df.to_csv(ticker+'.csv', index=False)
    print('       done...')
    print('-------------------------------------------')
    counter += 1

print('<<< COMPLETED !!! >>>')



>>> 1. Reading price data of EVER
       Exporting EVER.csv
       done...
-------------------------------------------
>>> 2. Reading price data of FSS
       Exporting FSS.csv
       done...
-------------------------------------------
>>> 3. Reading price data of ECL
       Exporting ECL.csv
       done...
-------------------------------------------
>>> 4. Reading price data of AYUD
       Exporting AYUD.csv
       done...
-------------------------------------------
>>> 5. Reading price data of AEONTS
       Exporting AEONTS.csv
       done...
-------------------------------------------
>>> 6. Reading price data of BBL
       Exporting BBL.csv
       done...
-------------------------------------------
>>> 7. Reading price data of TPOLY
       Exporting TPOLY.csv
       done...
-------------------------------------------
>>> 8. Reading price data of RBF
       Exporting RBF.csv
       done...
-------------------------------------------
>>> 9. Reading price data of SFLEX
       Exportin

## Getting SET Index Data from Broker MT5

In [1]:
############### Export SET Index ###############
# import libraries
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from IPython.display import display

# connecting to MetaTrader5
# mt5.initialize()
# if not mt5.initialize(path='C:\\Program Files\\Top Trader MetaTrader 5\\terminal64.exe'):
if not mt5.initialize(path='C:\\Program Files\\Top Trader MetaTrader 5-01\\terminal64.exe'):
# if not mt5.initialize(path='C:\\Program Files\\MetaTrader 5 IC Markets (SC)\\terminal64.exe'):
    print("initialize failed, Error code =", mt5.last_error())
    mt5.shutdown()

account="HIDDEN"
authorized=mt5.login(account, server="HIDDEN")

# requesting data
ohlc = mt5.copy_rates_range("SET", 
                            mt5.TIMEFRAME_D1, 
                            datetime(2020, 1, 1),
                            datetime.now())

# show ohlc data
# display(ohlc[:10])

# convert to DataFrame
ohlc_df = pd.DataFrame(ohlc)
ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
ohlc_df.drop(['tick_volume', 'spread'], inplace=True, axis=1)
ohlc_df.rename(columns = {'time':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'real_volume':'Volume'}, inplace = True)
ohlc_df['Adj Close'] = ohlc_df['Close'].copy()
ohlc_df = ohlc_df[['Date','Open','High','Low','Close','Adj Close','Volume']]

# show DataFrame
display(ohlc_df)

# save as csv using df.to_csv(filename.csv)
ohlc_df.to_csv('SET.csv', index=False)

# get number of symbols with symbols_total()
# num_symbols = mt5.symbols_total()
# num_symbols

# get symbol specifications
# symbol_info = mt5.symbol_info("ADVANC")._asdict()
# symbol_info

# get all symbols and their specifications
# symbols = mt5.symbols_get()
# for symbols in symbols:
# counter = 0
# for symbol in symbols[0:10] :
#     counter+=1
#     print(f'>>> {counter}. Reading price data of {symbol.name}')
#     # requesting data
#     ohlc = mt5.copy_rates_range(symbol.name, 
#                             mt5.TIMEFRAME_D1, 
#                             datetime(2021, 1, 1),
#                             datetime.now())
#     # convert to DataFrame
#     ohlc_df = pd.DataFrame(ohlc)
#     ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s')
#     ohlc_df.drop(['tick_volume', 'spread'], inplace=True, axis=1)
#     ohlc_df.rename(columns = {'time':'Date', 'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'real_volume':'Volume'}, inplace = True)
#     ohlc_df['Adj Close'] = ohlc_df['Close'].copy()
#     ohlc_df = ohlc_df[['Date','Open','High','Low','Close','Adj Close','Volume']]
    
#     # save as csv using df.to_csv(filename.csv)
#     print(f'       Exporting {symbol.name}.csv')
#     ohlc_df.to_csv(symbol.name+'.csv', index=False)
#     print('       done...')
#     print('-------------------------------------------')
print('<<< COMPLETED !!! >>>')



,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,1584.35,1597.51,1583.57,1595.82,1595.82,17737424730
1,2020-01-03,1596.95,1604.09,1593.02,1594.97,1594.97,18758835746
2,2020-01-06,1584.13,1585.36,1566.20,1568.50,1568.50,22621825700
3,2020-01-07,1578.52,1585.23,1570.06,1585.23,1585.23,20384416390
4,2020-01-08,1569.82,1571.84,1555.90,1559.27,1559.27,19789904721
...,...,...,...,...,...,...,...
614,2022-07-21,1538.40,1547.68,1535.72,1546.31,1546.31,2185599661834
615,2022-07-22,1551.20,1556.18,1542.32,1552.73,1552.73,2742947071762
616,2022-07-25,1553.70,1563.08,1548.18,1560.31,1560.31,2613973272132
617,2022-07-26,1563.16,1563.34,1552.51,1553.18,1553.18,2315756554307


<<< COMPLETED !!! >>>


## Saving data to csv

In [11]:
# save as csv using df.to_csv(filename.csv)
# ohlc_df.to_csv('eurusd_daily.csv', index=False)
ohlc_df.to_csv('ADVANC.csv', index=False)

## Read data from csv

In [25]:
# use pd.read_csv(filename)
ohlc_df2 = pd.read_csv('eurusd_daily.csv')
ohlc_df2

,time,open,high,low,close,tick_volume,spread,real_volume
0,2021-01-04,1.22393,1.23096,1.22276,1.22521,103051,0,0
1,2021-01-05,1.22484,1.23058,1.22434,1.22986,95295,0,0
2,2021-01-06,1.22941,1.23495,1.22657,1.23263,142869,0,0
3,2021-01-07,1.23252,1.23444,1.22452,1.22693,106299,0,0
4,2021-01-08,1.22711,1.22847,1.21932,1.22193,120479,0,0
...,...,...,...,...,...,...,...,...
233,2021-11-25,1.11966,1.12298,1.11956,1.12087,28599,0,0
234,2021-11-26,1.12058,1.13308,1.12042,1.13087,75257,0,0
235,2021-11-29,1.13014,1.13124,1.12584,1.12918,56344,0,0
236,2021-11-30,1.12905,1.13829,1.12354,1.13386,76583,0,0


## Saving data to Database (SQLite)

In [28]:
# importing database library
import sqlite3

# create the database / connect to the database
con = sqlite3.connect('traderpy.db')

# save data to database using df.to_sql(tablename)
ohlc_df.to_sql('eurusd_d1', con=con, if_exists='replace', index=False)

# tip: if_exists has following options: 'replace', 'append'

## Query Data from DataBase

In [29]:
# use pd.read_sql
ohlc_df3 = pd.read_sql('select * from eurusd_d1', con=con)
ohlc_df3

,time,open,high,low,close,tick_volume,spread,real_volume
0,2021-01-04 00:00:00,1.22393,1.23096,1.22276,1.22521,103051,0,0
1,2021-01-05 00:00:00,1.22484,1.23058,1.22434,1.22986,95295,0,0
2,2021-01-06 00:00:00,1.22941,1.23495,1.22657,1.23263,142869,0,0
3,2021-01-07 00:00:00,1.23252,1.23444,1.22452,1.22693,106299,0,0
4,2021-01-08 00:00:00,1.22711,1.22847,1.21932,1.22193,120479,0,0
...,...,...,...,...,...,...,...,...
233,2021-11-25 00:00:00,1.11966,1.12298,1.11956,1.12087,28599,0,0
234,2021-11-26 00:00:00,1.12058,1.13308,1.12042,1.13087,75257,0,0
235,2021-11-29 00:00:00,1.13014,1.13124,1.12584,1.12918,56344,0,0
236,2021-11-30 00:00:00,1.12905,1.13829,1.12354,1.13386,76583,0,0


In [30]:
# use query with conditions
ohlc_df4 = pd.read_sql('select * from eurusd_d1 where time >= "2021-11-01"', con=con)
ohlc_df4

,time,open,high,low,close,tick_volume,spread,real_volume
0,2021-11-01 00:00:00,1.15558,1.16092,1.15460,1.16066,43399,0,0
1,2021-11-02 00:00:00,1.16048,1.16134,1.15753,1.15793,43983,0,0
2,2021-11-03 00:00:00,1.15790,1.16164,1.15624,1.16127,51379,0,0
3,2021-11-04 00:00:00,1.16116,1.16165,1.15284,1.15545,47924,0,0
4,2021-11-05 00:00:00,1.15518,1.15733,1.15134,1.15675,51057,0,0
5,2021-11-08 00:00:00,1.15602,1.15950,1.15506,1.15873,40856,0,0
6,2021-11-09 00:00:00,1.15852,1.16089,1.15701,1.15944,46947,0,0
7,2021-11-10 00:00:00,1.15925,1.15953,1.14761,1.14767,59078,0,0
8,2021-11-11 00:00:00,1.14783,1.14877,1.14430,1.14493,39593,0,0
9,2021-11-12 00:00:00,1.14500,1.14622,1.14330,1.14438,40787,0,0


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime
from IPython.display import display

# connecting to MetaTrader5
# mt5.initialize()
# if not mt5.initialize(path='C:\\Program Files\\Top Trader MetaTrader 5\\terminal64.exe'):
if not mt5.initialize(path='C:\\Program Files\\Top Trader MetaTrader 5-01\\terminal64.exe'):
# if not mt5.initialize(path='C:\\Program Files\\MetaTrader 5 IC Markets (SC)\\terminal64.exe'):
    print("initialize failed, Error code =", mt5.last_error())
    mt5.shutdown()

account="HIDDEN"
authorized=mt5.login(account, server="HIDDEN")

tickers_df = pd.DataFrame()
symbols = mt5.symbols_get()
for symbol in symbols :
    print(symbol.name)
    # tickers.append(symbol.name)
    # tickers.loc[symbol] = symbol.name
    tickers_df = tickers_df.append({'Symbol': symbol.name}, ignore_index=True)
# tickers
tickers_df.to_csv('Tickers.csv', index=False)